will learn:
1. Agent Workflows
2. Use of tools to call functions
3. Agents collaboration via Tools and Handoffs

In [9]:
import os
import json
import requests
import asyncio
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY is not set in the environment variables.")
else:
    print(f"OpenAI API exists: {openai_api_key[:10]}...")

OpenAI API exists: sk-proj-Yq...


In [2]:
TESTMAIL_API_KEY = os.getenv("TESTMAIL_API_KEY")
NAMESPACE = "s65af"
BASE_URL = "https://api.testmail.app/api/json"

if not TESTMAIL_API_KEY:
    raise ValueError("TESTMAIL_API_KEY is not set in the environment variables.")
else:
    print(f"TestMail API exists: {TESTMAIL_API_KEY[:10]}...")

TestMail API exists: 94e4599d-a...


### Step 1: Agent workflow

In [3]:
instructions1 = """You are a sales agent working for CompIAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails."""

instructions_2 = """
You are a humorous, engaging sales agent working for CompIAI.
CompIAI provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
Write witty, engaging cold emails that are likely to get a response.
"""

instructions_3 = """
You are a busy sales agent working for CompIAI.
CompIAI provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI.
Write concise, to the point cold emails.
"""

In [4]:
sales_agent_1 = Agent(
    name="CompIAI Sales Agent 1",
    instructions=instructions1,
    model="gpt-5-nano",
)

sales_agent_2 = Agent(
    name="CompIAI Sales Agent 2",
    instructions=instructions_2,
    model="gpt-5-nano",
)

sales_agent_3 = Agent(
    name="CompIAI Sales Agent 3",
    instructions=instructions_3,
    model="gpt-5-nano",
)

In [5]:
result = Runner.run_streamed(sales_agent_1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Accelerate SOC 2 readiness with AI-powered automation

Hi [FirstName],

I know SOC 2 readiness is a top priority for growing teams like [Company], especially with audit deadlines and last-minute evidence requests. CompIAI is an AI-powered SaaS designed to simplify SOC 2 compliance and audit prep.

How we help:
- Automated evidence collection and control mapping aligned to SOC 2 Trust Services Criteria
- Ready-made policy templates and audit-ready evidence packages
- Continuous monitoring and remediation tracking to keep controls in check
- A centralized, audit-ready dashboard that speeds reviews

Benefits you’ll feel:
- Less manual work and fewer errors from scattered data
- Faster audit cycles and clearer progress visibility
- Easier demonstrations of ongoing compliance to auditors and regulators

Would you be open to a 15-minute call next week to discuss your current SOC 2 plan and see a quick demo tailored to your controls and tech stack?

Best regards,
[Your Name]
[Title]


## Concurrent execution of multiple agents

In [6]:
message = "Write a cold sales email"
with trace("Concurrent Sales Agents: cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Simplify SOC 2 readiness for [Company] with AI-powered audit prep

Hi [FirstName],

As [Company] scales and handles more customer data, SOC 2 compliance becomes essential—and time-consuming. CompIAI offers an AI-powered SaaS platform that automates evidence collection, control testing, and remediation tracking to deliver continuous SOC 2 readiness.

What you get with CompIAI:
- Auto-map controls to the Trust Services Criteria (security, availability, processing integrity, confidentiality, privacy)
- End-to-end evidence generation and a centralized artifacts repository
- Continuous monitoring with alerts for gaps and policy drift
- Seamless integrations with AWS/Azure/GCP and common DevOps tools
- Faster audit prep and reduced manual workload

Would you be open to a 20-minute demo this week to show how it would fit [Company]’s control environment and upcoming audit cycle? I can tailor the session to the controls most relevant to you.

Best regards,
[Your Name]
[Title], CompIAI


In [7]:
sales_picker_agent = Agent(
    name="Sales Email Picker Agent",
    instructions="""
    Pick the best cold sales email from the given options.
    Imagine you are a customer and pick the one you are most likely to respond to.
    Do not give an explanation; reply with the selected email only.
    """,
)

In [8]:
message = "write a cold sales email"
with trace("Sales Email Picker Agent: pick best email"):
    results = await asyncio.gather(
        Runner.run(sales_agent_1, message),
        Runner.run(sales_agent_2, message),
        Runner.run(sales_agent_3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n" + "\n\n".join(outputs)

    best_email = await Runner.run(sales_picker_agent, emails)

    print(f"Best email:\n{best_email.final_output}")

Best email:
**Option 2: Data-driven and concise**

Subject: See how AI trims SOC 2 prep time by 40–60%

Body:
Hi [Name], quick question: what would a 40–60% reduction in SOC 2 prep time buy your team this quarter? At CompIAI, we automate evidence gathering, control mapping, and artifact generation to deliver an audit-ready package faster.

What you’ll get:
- Centralized evidence library with auto-collection
- AI-guided control mapping and gap detection
- Continuous controls monitoring and drift alerts
- Ready-made policy templates and risk assessments
- One-click, audit-ready package with executive summary

If you’re evaluating SOC 2 readiness tools, I’d love to show you a 15-minute walkthrough tailored to your controls. Are you available Tuesday or Thursday? Best, [Your Name]


### Tools and agent Instruction

In [10]:
@function_tool
def send_email(body: str):
    """Send an email to CompIAI."""
    params = {
        'apikey': os.getenv("TESTMAIL_API_KEY"),
        'namespace': os.getenv("TESTMAIL_NAMESPACE"),
        'livequery': 'true'  # Enable live queries
    }
    try:
        response = requests.get(url=BASE_URL, params=params)
        response.raise_for_status()

        data = response.json()
        print(json.dumps(data, indent=4))

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

In [11]:
send_email

FunctionTool(name='send_email', description='Send an email to CompIAI.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x104c0d440>, strict_json_schema=True, is_enabled=True)

In [12]:
print(send_email)

FunctionTool(name='send_email', description='Send an email to CompIAI.', params_json_schema={'properties': {'body': {'title': 'Body', 'type': 'string'}}, 'required': ['body'], 'title': 'send_email_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x104c0d440>, strict_json_schema=True, is_enabled=True)
